In [1]:
import math
import random
from sympy import isprime

# Part 1

* Create a phrase of about 45 letters, including your name.
* Convert to a number as follows: A = 01, . . . , Z = 26, space = 27 etc.
Concatenating the digits, you will get a number of about 90 digits.
* Add 10 random digits at the end to obtain an odd 100-digit number and test if n is a
prime number. Change these last 10 digits until n is prime.

In [16]:
# Create a phrase of about 45 letters, including your name.
phrase = ' ACHITA IS BULLISH ON METIS AND HYBRID ROLLUP'

In [17]:
len(phrase)

45

In [18]:
def char_to_number(s):
    result = ""

    # Concatenating the digits, you will get a number of about 90 digits.
    for char in s.upper():
        # Convert to a number as follows: A = 01, . . . , Z = 26, space = 27 etc.
        if 'A' <= char <= 'Z':
            result += f"{ord(char) - 64:02}"
        elif char == ' ':
            result += "27"
        else:
            continue

    return result

In [19]:
num_phrase = char_to_number(phrase)
num_phrase

'270103080920012709192702211212091908271514271305200919270114042708250218090427181512122116'

In [20]:
def generate_odd_100_digit_number(num):
    while True:
        # Add 10 random digits at the end to obtain an odd 100-digit number and test if n is a prime number.
        random_digits = ''.join([str(random.randint(0, 9)) for _ in range(9)]) + str(random.choice([1, 3, 5, 7, 9]))
        candidate = int(num + random_digits)

        if len(str(candidate)) == 100 and candidate % 2 != 0:
            return candidate

def find_100_digit_prime(num):
    while True:
        # Change these last 10 digits until n is prime.
        candidate = generate_odd_100_digit_number(num)
        if isprime(candidate):
            return candidate

In [21]:
## prime_number = 2425260103080920011702092123060417060502101115161801190518202301180505150702102304100220115438909631
prime_number = find_100_digit_prime(num_phrase)
print(f"Prime number: {prime_number}")

Prime number: 2701030809200127091927022112120919082715142713052009192701140427082502180904271815121221166481104813


In [ ]:
#prime_number = 2701030809200127091927022112120919082715142713052009192701140427082502180904271815121221166481104813

# Part 2

* You can use existing libraries to do computations on large numbers, but you must
implement your own version of the Miller-Rabin algorithm, according to Fact 4.24 in the
HAC (Chapter 4, Page 139). You should hand in the text of this function.

In [77]:
def find_exponent_of_2(n):
    if n % 2 == 0:
        return "n must be odd"

    n_minus_1 = n - 1
    exponent_of_2 = 0

    while n_minus_1 % 2 == 0:
        n_minus_1 //= 2
        exponent_of_2 += 1
    return exponent_of_2

def find_s_and_r(n):
    s = find_exponent_of_2(n)
    r = (n-1)//(2**s)
    return s, r

In [70]:
# Algorithm 2.143: Repeated square-and-multiply algorithm for exponentiation in Z_n
def repeated_square_and_multiply(a, k, n):
  '''
  Input:
  a - Z_n
  0 <= k < n, where k = Sum^{t}_{i=0} k_i*2^i

  Output:
  a**k % n
  '''
  if k < 0:
    return 'k must greater than or equal to 0'
  if k >= n:
    return 'k must less than n'

  binary_k = bin(k)[2:][::-1]

  # Step 1
  b = 1
  if k == 0:
    return b

 # Step 2
  A = a

  #Step 3
  if binary_k[0] == '1':
    b = a

  #Step 4
  for i in range (1, len(binary_k)):
    A = A**2 % n
    if binary_k[i] == '1':
      b = A*b % n

  return b

In [110]:
def MILLER_RABIN(n, t):
  '''
  Input:
  n - odd number >= 3
  t - security parameter t >= 1
  Output:
  Check whether n is prime?
  '''
  # Step 1
  s,r = find_s_and_r(n)
  # Step 2
  for i in range(1, t):
    # Step 2.1
    a = random.randint(2, n-2)
    # Step 2.2
    y = repeated_square_and_multiply(a, r, n)
    if y!= 1 and y!= n-1:
      j  = 1
      while j <= s - 1 and y != n-1:
        y = y**2 % n
        if y == 1:
          return 'composite'
        j += 1
      if y!= n-1:
        return 'composite'
    # Step 3
    return 'prime'

In [111]:
t = 10
MILLER_RABIN(prime_number, t)

'prime'

# Part 3

* If you test only one $a \in \mathbb{Z}^{*}_{n}$, what is the probability of error according to Table 4.3 of the HAC? (Chapter 4, Page 147)

In [98]:
n = prime_number
k = len(bin(n)[2:])
print(k)

331


In [ ]:
t = 10

In [106]:
# From 4.48
def compute_upper_bounds_p(k, t):
  if (t == 1) and (k >= 2):
    return k**2 * 4**(2-k**(1/2))
  elif ((t == 2) and (k >= 88)) or ((k/9 >= t >= 3) and (k >= 21)):
    return k**(3/2) * 2**t * t**(-1/2) * 4**(2-(t*k)**(1/2))
  elif (k/4 >= t >= k/9) and (k >= 21):
    return (7/20) * k * (2**(-5*t)) + (1/7) * (k**(15/4)) * (2**(-k/(2 - 2*t))) +12 * k * (2**(-k/4 - 3*t))
  elif (t >= k/4) and (k >= 21):
    return 1/7 * (k**(15/4)) * (2**(-k/2 - 2*t))
  else:
    return 'error'

In [109]:
# Compute the exponential of p_{k, t} with base 1/2
math.log(compute_upper_bounds_p(k, t), 1/2)

90.17013184637307

Reference

*   https://drive.google.com/file/d/1Wi4E6cg2QazGZgvkWlDC3xExByfhg71d/view?pli=1
*   https://docs.google.com/forms/d/e/1FAIpQLSc2VBg2l6ONCXUJGC88_CvnzTKAtX42l4xfa5n4wRr-OlyP3g/viewform